<a href="https://colab.research.google.com/github/ferdonof/topicos-2/blob/master/calculadora_sueldos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Programa de cálculo de sueldos**
## 1 - Obtención de los datos

Los datos utilizados son los mismos que se utilizaron en el ejemplo de clase.

In [8]:
!git clone https://github.com/gabyaleperez/redesNeuronales.git

fatal: destination path 'redesNeuronales' already exists and is not an empty directory.


In [9]:
import numpy as np
import pandas as pd

filename = './redesNeuronales/datos_empleados_50000.csv'
data = pd.read_csv(filename, index_col=0)
print(data)


       sueldo_basico categoria  ausencias  cantidad_hijos  sueldo
0              33600         C          2               2   37600
1              14200         A          0               0   15570
2              10600         C          2               1   13850
3              16300         C          0               1   19620
4              20900         C          2               1   24150
...              ...       ...        ...             ...     ...
49995          22500         C          2               2   26500
49996          70600         A          1               0   71900
49997          23800         B          1               3   28050
49998          44800         A          1               3   48350
49999          57100         A          2               1   59150

[50000 rows x 5 columns]


## 2 - Procesamiento de los datos
Homogeneizar los datos, separarlos en  etiquetas para poder utilizalos de entrada.

In [10]:
from sklearn.model_selection import train_test_split

#Clasificación de los datos utilizando onehot
data = pd.get_dummies(data) 

#separamos el sueldo, para que no se use comoparte del entrenamiento
x = data.drop(['sueldo'], axis=1) 

y = np.array(data['sueldo'])

# utilizo la libreria para separar los datos de entrenamiento de los de testing
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.2)

from pprint import pprint

# muestro los datos de entrenamiento (el 80% de los datos será para entrenar la red)
pprint(train_x)

       sueldo_basico  ausencias  cantidad_hijos  categoria_A  categoria_B  \
26302          54100          0               2            0            0   
14006          69200          2               2            0            1   
49089          61900          2               3            0            1   
37797          79900          1               3            0            0   
5649           38900          1               0            0            1   
...              ...        ...             ...          ...          ...   
17921          13300          2               3            0            0   
3752           45200          2               0            0            0   
47493          75300          1               2            1            0   
36756          59800          1               3            0            0   
30105          25400          2               1            0            1   

       categoria_C  
26302            1  
14006            0  
49089       

## 3 - Armado de la RN
Al tener 6 columnas, voy a necesitar 6 neuronas de entrada. Como la salida va a ser el sueldo (estimado) del empleado, voy a necesitar una neurona para la salida.

In [11]:
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Nadam

model = Sequential()

# capa de entrada
model.add(Dense(30, input_shape = (6, ), activation='relu', kernel_initializer='uniform'))

# capas intermedia
# model.add(Dense(25, activation='relu', kernel_initializer='random_normal'))
# model.add(Dense(15, activation='relu', kernel_initializer='random_normal'))

# capa de salida
model.add(Dense(1, activation='relu'))


# hiperparámetros
loss_function = 'mean_squared_error' 
learning_rate = 0.001


model.compile(loss = loss_function, optimizer=Nadam(learning_rate = learning_rate))

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 30)                210       
                                                                 
 dense_3 (Dense)             (None, 1)                 31        
                                                                 
Total params: 241
Trainable params: 241
Non-trainable params: 0
_________________________________________________________________


## 4 - Entrenamiento de la red

In [12]:
batch_size = 10000
epochs = 200

model.fit(train_x, train_y, verbose=2, batch_size = batch_size, epochs = epochs)



Epoch 1/200
4/4 - 1s - loss: 3372401920.0000 - 634ms/epoch - 159ms/step
Epoch 2/200
4/4 - 0s - loss: 3372402176.0000 - 23ms/epoch - 6ms/step
Epoch 3/200
4/4 - 0s - loss: 3372401920.0000 - 23ms/epoch - 6ms/step
Epoch 4/200
4/4 - 0s - loss: 3372402176.0000 - 26ms/epoch - 6ms/step
Epoch 5/200
4/4 - 0s - loss: 3372401920.0000 - 25ms/epoch - 6ms/step
Epoch 6/200
4/4 - 0s - loss: 3372401920.0000 - 27ms/epoch - 7ms/step
Epoch 7/200
4/4 - 0s - loss: 3372401920.0000 - 25ms/epoch - 6ms/step
Epoch 8/200
4/4 - 0s - loss: 3372401920.0000 - 24ms/epoch - 6ms/step
Epoch 9/200
4/4 - 0s - loss: 3372402176.0000 - 21ms/epoch - 5ms/step
Epoch 10/200
4/4 - 0s - loss: 3372401920.0000 - 26ms/epoch - 6ms/step
Epoch 11/200
4/4 - 0s - loss: 3372401920.0000 - 27ms/epoch - 7ms/step
Epoch 12/200
4/4 - 0s - loss: 3372402176.0000 - 28ms/epoch - 7ms/step
Epoch 13/200
4/4 - 0s - loss: 3372401920.0000 - 22ms/epoch - 6ms/step
Epoch 14/200
4/4 - 0s - loss: 3372401920.0000 - 33ms/epoch - 8ms/step
Epoch 15/200
4/4 - 0s - lo

## 5 - Evaluar el resultado

In [13]:
result = model.evaluate(test_x, test_y)

313/313 [==============================] - 1s 1ms/step - loss: 3329243136.0000


## 6 - Predicción de los nuevos datos

In [14]:
pred_y = model.predict(test_x)
  
print("Resultados de las predicciones\n")  
print("|\tEsperado $\t|\tObtenido $|")
print("-------------------------------------------------")

for i in range(5):
  print("|\t",test_y[i],"\t\t|\t",pred_y[i][0],"|")


313/313 [==============================] - 0s 1ms/step
Resultados de las predicciones

|	Esperado $	|	Obtenido $|
-------------------------------------------------
|	 16570 		|	 0.0 |
|	 47800 		|	 0.0 |
|	 65170 		|	 0.0 |
|	 57470 		|	 0.0 |
|	 52820 		|	 0.0 |
